# Welcome to the Synergy Oneliners
Created by Fredrik Tärnell - 2019-02-20<br>

This notebook contains useful oneliners for particular occations.<br>
**Don't use them without fully understanding them**.<br> Some will wipe resources away no questions asked.

Jupyter Notebooks can be found: https://github.com/frippe75/synergy-jupyter-notebooks<br>
Together with a Docker image with all dependencies included and a quickstart guide.


### API information

http://h17007.www1.hpe.com/docs/enterprise/servers/oneview4.1/cic-api/en/index.html <br>



### Python specific
https://hewlettpackard.github.io/python-hpOneView/index.html<br>
https://github.com/HewlettPackard/python-hpOneView<br>

<br>

## Table of content

- **[Networks](#Network)**<br>
   + [List all networks](#Network-show)<br>
   + [Create a single network](#Network-create)<br>
   + [Delete a single network](#Network-delete)<br>
   + [Create multiple networks](#Network-bulk-create)<br>
   + [Delete multiple networks](#Network-bulk-delete)<br>
   + [Delete ALL networks](#Network-delete-all)<br>
   + [Create the netop user](#Netop)<br>
- **[Server Profiles](#SP)**<br>
   + [Power On/Off all Profiles](#SP-power)<br>
   + [Delete all Profiles](#SP-delete)<br>
   + [Server Profile Mobility](#SP-mobility)<br>
- **[Server Profiles Templates](#SPT)**<br>
   + [Delete all Profiles Templates](#SPT-delete)<br>
- **[Volume Templates](#VolumeTemplates)**<br>
   + [Delete all volume templates](#VolumeTemplates-deleteall)<br>
- **[Volumes](#Volumes)**<br>
   + [Delete all volumes](#Volume-deleteall)<br>
- **[Appliances](#Appliances)**<br>
   + [eFuse Composer / Image Streamer](#Appliance-efuse)<br>
   + [eFuse Compute module](#Appliance-efuse-compute)<br>
<br><br>
# First section - Simple tasks via Python



### Import the python OneView library with some additional dependencies for this playbook

In [17]:
from hpOneView.oneview_client import OneViewClient
import pandas, paramiko
from IPython.display import display, Markdown, Javascript
from pprint import pprint

# Simple helper function to make nicer tables using markdown and the pandas lib
# TODO: get it into helper module that gets imported above 
def my_table(source=dict(), resource="Resource", columns='name'):
    data = pandas.DataFrame.from_dict(source)
    table = data.to_html(index=False,header=True,columns=columns)
    
    display(Markdown('### ' + resource + ' in Synergy Composer at https://' + config['ip']))
    display(Markdown(table))
    
def ssh_command(self, command):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    # Make connection and create shell.
    client.connect(self.ipaddr, self._sshtun_port, self.ssh_user, self.ssh_user_pass)
    shell = client.invoke_shell()

    # Execute command and get results.
    _, stdout, stderr = client.exec_command(command)
    data = self._read_command_output(stdout, stderr, 'both')

    # Close connection.
    shell.close()
    client.close()

    return data 

### Configure your environment

In [3]:
composer_ip = "synergy.hpedemo.local"

config = {
    "api_version": "1000",
    "ip": composer_ip,
    "credentials": {
        "userName": "Administrator",
        "authLoginDomain": "local",
        "password": '********'
    }
}
pprint(config)

{'api_version': '1000',
 'credentials': {'authLoginDomain': 'local',
                 'password': 'HPEc0nverged!',
                 'userName': 'Administrator'},
 'ip': 'synergy.hpedemo.local'}


<br><br>
## Login against the Appliance / Composer instance:
Create new object "ov" with the config (see above) using the JSON config dict above

In [4]:
try:
    ov = OneViewClient(config)
    print("\nLogged in against " + config['ip'] + " successfully\n")
except HPOneViewException as e:
    print("\nLogged in failed:")
    print(e.msg)


Logged in against synergy.hpedemo.local successfully



<br><br>
# Networking  <a name="Network"></a>
## Show existing networks  <a name="Network-show"></a>
This example uses Pandas dataframe to create the HTML table<br>
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_html.html?highlight=to_html#pandas.DataFrame.to_html


In [ ]:
poc_networks = ov.ethernet_networks.get_all()

#print(poc_networks)

if poc_networks:
    my_table(poc_networks, resource="Subnets", columns=['name','description','vlanId','uri','purpose'])


<br><br>
## Create a network  <a name="Network-create"></a>

In [ ]:
options = {
    "name": "New network",
    "vlanId": 3000,
    "ethernetNetworkType": "Tagged",
    "purpose": "General",
    "smartLink": False,
    "privateNetwork": False,
    "type" : "ethernet-networkV4",
    "connectionTemplateUri": None
}

ethernet_network = ov.ethernet_networks.create(options)
print("Created ethernet-network " + ethernet_network['name'] + "   URI: " + ethernet_network['uri'] + "\nsuccessfully.")

<br><br>
## Delete a network  <a name="Network-delete"></a>

In [ ]:
ov.ethernet_networks.delete(ethernet_network)
print("Delete the network created above: " + ethernet_network['name'])

<br><br>
## Create bulk networks <a name="Network-bulk-create"></a>

In [12]:
options_bulk = {
    "vlanIdRange": "11-20",
    "purpose": "General",
    "namePrefix": "Net",
    "smartLink": False,
    "privateNetwork": False,
    "type": "bulk-ethernet-networkV1",
    "bandwidth": {
        "maximumBandwidth": 10000,
        "typicalBandwidth": 2000
    }
}

# create the networks
ethernet_nets_bulk = ov.ethernet_networks.create_bulk(options_bulk)

# simply display whats exepected
for net in ethernet_nets_bulk: 
  print("created network: " + net['name'] + " URI: " + net['uri'])

created network: Net_11 URI: /rest/ethernet-networks/c5e06b30-38ce-4610-aba5-e87ee9be90bc
created network: Net_12 URI: /rest/ethernet-networks/5d06575a-43d6-4a94-8880-1c0bb0f518c1
created network: Net_13 URI: /rest/ethernet-networks/04587364-71bc-4123-825f-a1c63a972103
created network: Net_14 URI: /rest/ethernet-networks/1ba54684-2385-4602-a8a7-64d77d03aa2f
created network: Net_15 URI: /rest/ethernet-networks/a247beb9-218f-43c9-8704-e8352564aad9
created network: Net_16 URI: /rest/ethernet-networks/9a696465-363a-4c36-b169-d58b8e7d2b00
created network: Net_17 URI: /rest/ethernet-networks/aa27fcef-1fc3-4d9c-aa16-89eaaa9c59c2
created network: Net_18 URI: /rest/ethernet-networks/98803094-1ddf-4b39-98ee-bc47b79fc444
created network: Net_19 URI: /rest/ethernet-networks/374ec04d-822e-4fbb-9cb0-d1bfe73660fb
created network: Net_20 URI: /rest/ethernet-networks/0af1a292-3e3a-4153-9663-065f70e7fa51


<br><br>
## Delete the bulks network above  <a name="Network-bulk-delete"></a>

In [5]:
for net in ethernet_nets_bulk:
    ov.ethernet_networks.delete(net)
    print("delete network: " + net['name'])

NameError: name 'ethernet_nets_bulk' is not defined

<br><br>
## Delete ALL networks in Oneview  <a name="Network-delete-all"></a>
This will **delete ALL networks off all types** so don't run unless you mean it!

In [11]:
all_nets = ov.ethernet_networks.get_all()
#pprint(all_nets)

for net in all_nets:
    # If the ov command below is commented out it's safe to give a try as a dry-run
    ov.ethernet_networks.delete(net)
    print("Deleting network: " + net['name'])

<br><br>
## Create the netop user which was removed part of 4.2 release  <a name="Netop"></a>


In [23]:
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

# https://github.com/paramiko/paramiko/issues/1386
import warnings
warnings.filterwarnings(action='ignore',module='.*paramiko.*')

# Make connection and create shell.
print("Connecting via ssh")
try:
    client.connect("10.10.20.141", 22, 'netop', "netoppwd")
    #client.connect("10.0.2.15", 22, 'frta', "frippe75")
    #shell = client.invoke_shell()
except Exception as e:
    print('Connection Failed')
    print(e)

# Execute command and get results.
print("Executing command ")
_, stdout, stderr = client.exec_command("show serial-number")
_, stdout, stderr = client.exec_command("show serial-number")
print("command executed")
print(len(stdout.channel.in_buffer))
stdout_str = (stdout.channel.recv(len(stdout.channel.in_buffer)))
#exit_status = stdout.channel.recv_exit_status()
print(stdout.read())
_, stdout, stderr = client.exec_command("show serial-number; exit")
print("command executed")
#for line in stdout.readlines():
#    print(line)
#data = self._read_command_output(stdout, stderr, 'both')

Connecting via ssh
Executing command 
command executed
0
b'\r\n\r\x1b[100B\r\n\t\t \r\n\t\t    Virtual Connect SE 40Gb F8 Module for Synergy\r\n\t\tType "show system acknowledgement" to see the licenses\r\n\t\tfor the open source software used by this product.\r\n\t\t \r\n\r\n\rOneView# \r\nIdle Timer expired, Timing Out !!!\n\x1b\r                       \rConnection closed by foreign host\r\n\r\nEntering character mode\r\nEscape character is \'^]\'.\r\n'
command executed


In [4]:
# set the password
netop_pwd = 'netoppwd'
ssh_cmd = '"show serial-number"'

# https://github.com/paramiko/paramiko/issues/1386
import warnings
warnings.filterwarnings(action='ignore',module='.*paramiko.*')
    
# loop over all VC40F8 modules
ics = ov.interconnects.get_all()
for ic in ics:
    if ic['model'] == "Virtual Connect SE 40Gb F8 Module for Synergy":
        print("Going to add the netop user on the interconnect with uri " + ic['uri'])
        #print(ic['uri'])
        updated_interconnect = ov.interconnects.patch(ic['uri'],
                                                        'replace',
                                                        '/netOpPasswd',
                                                        netop_pwd)
        for ip in ic['ipAddressList']:
            if ip['ipAddressType'] == 'Ipv4Static': 
                ipv4 = ip['ipAddress']
                
                print("Ip address of the VC module is: " + ipv4)
                client = paramiko.SSHClient()
                client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

                # Make connection and create shell.
                print("Connecting via ssh to run " + ssh_cmd + "")
                try:
                    client.connect(ipv4, 22, 'netop', netop_pwd)
                    #shell = client.invoke_shell()
                except Exception as e:
                    print('Connection Failed')
                    print(e)

                # Execute command and get results.
                print("Executing command " + ssh_cmd)
                _, stdout, stderr = client.exec_command(ssh_cmd)
                for line in stdout.readlines():
                    print(line)
                #data = self._read_command_output(stdout, stderr, 'both')

                # Close connection.
                #shell.close()
                client.close()


Going to add the netop user on the interconnect with uri /rest/interconnects/1385b2a6-9c01-4577-b841-c8e4dbf7b16c
Ip address of the VC module is: 10.10.20.141
Connecting via ssh to run "show serial-number"


Exception: Error reading SSH protocol banner[Errno 104] Connection reset by peer
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/paramiko/transport.py", line 2138, in _check_banner
    buf = self.packetizer.readline(timeout)
  File "/opt/conda/lib/python3.7/site-packages/paramiko/packet.py", line 367, in readline
    buf += self._read_timeout(timeout)
  File "/opt/conda/lib/python3.7/site-packages/paramiko/packet.py", line 561, in _read_timeout
    x = self.__socket.recv(128)
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/paramiko/transport.py", line 1966, in run
    self._check_banner()
  File "/opt/conda/lib/python3.7/site-packages/paramiko/transport.py", line 2143, in _check_banner
    "Error reading SSH protocol banner" + str(e)
paramiko.ssh_exception.SSHException: Error reading SSH prot

Connection Failed
Error reading SSH protocol banner[Errno 104] Connection reset by peer
Executing command "show serial-number"


SSHException: SSH session not active

<br><br>
## Server Profile Templates <a name="SPT"></a>

### Delete ALL existing Server Profile Templates <a name="SPT-delete"></a>

WARNING: Will delete ALL profiles templates unconditionally!

In [ ]:
# get a list of all the spts
spts = ov.server_profile_templates.get_all()

# delete them one-by-one
for spt in spts: 
  print("Deleting the Server Profile Template: " + spt['name'] )
  ov.server_profile_templates.delete(spt)


<br><br>
## Server Profiles <a name="SP"></a>

### Delete ALL existing Server Profiles <a name="SP-delete"></a>

WARNING: Will delete ALL profiles unconditionally!

In [ ]:
# get a list of all the sp's
sps = ov.server_profiles.get_all()

# delete them one-by-one
for sp in sps: 
  print("Deleting the Server Profile: " + sp['name'] )
  ov.server_profiles.delete(sp)


### Power on/off  ALL existing Server Profiles <a name="SP-power"></a>
Function power_off_profile is a helper function since it will be used for both powering off all servers as well as for the server profile mobility use case.

In [29]:
# takes a server profile and a 'On' or 'Off' for new_state as input 
def change_profile_power(sp, new_state):
    configuration = {
            "powerState": new_state,
            "powerControl": "MomentaryPress"
    }
    # print them nicely
    # pprint(sp)
    sp_uri = sp['serverHardwareUri']
    sp_name = sp['name']
    # remember some profiles be unassigned. i.e they dont have a hardware uri
    if sp_uri is not None:
      print("Powering " + new_state + " the Server Profile: " + sp_name + " using ServerHardwareUri " +  sp_uri ) 
      try:
          ov.server_hardware.update_power_state(configuration,  sp_uri)
          print("Successfully changed the power state of '" + sp_name + "' to " + configuration['powerState'])
      except HPOneViewException as e:
          print(e.msg)
    else:
      print("Server Profile: " + sp_name + " is unassigned")

**Warning:** Will power on or off all profiles depending on the new_state variable.<br>
The actual command is commented out for safety reasons.

In [28]:
# what powerstate to achieve
new_state = "On" # it's either "On" or "Off"

# get a list of all the sp's
sps = ov.server_profiles.get_all()

# power them OFF them one-by-one
for sp in sps: 
    # change_profile_power(sp, new_state)
      

Powering On the Server Profile: CentOS-Golden using ServerHardwareUri /rest/server-hardware/36343537-3338-4E43-3736-303230355A46
Successfully changed the power state of 'CentOS-Golden' to On
Powering On the Server Profile: Syn-Cluster1 using ServerHardwareUri /rest/server-hardware/39313738-3534-5A43-4A38-343330373357
Successfully changed the power state of 'Syn-Cluster1' to On
Powering On the Server Profile: Syn-Cluster2 using ServerHardwareUri /rest/server-hardware/39313738-3534-5A43-4A38-343330373356
Successfully changed the power state of 'Syn-Cluster2' to On


In [18]:
# trick to run the cell above via code. To included a needed function for instance
Javascript("Jupyter.notebook.execute_cells(IPython.notebook.get_selected_index()-1)")

<IPython.core.display.Javascript object>

### Server Profile Mobility <a name="SP-mobility"></a>
**Warning:** Will power off the profile depending on current state

In [39]:
# Move profile CentOs-Golden from existing hardware to new_server
sp_name = 'CentOS-Golden'
new_server = 'HPEDemo-POCframe01, bay 3'
force_power_off = True

sp = ov.server_profiles.get_by_name(sp_name)
# check that the profile is currently assigned
if sp['serverHardwareUri'] is not None:
    print("Profile " + sp_name + " is currently assigned will unassign it.")
    if force_power_off:
        change_profile_power(sp,"Off")
        #pprint(profile)
    sp['serverHardwareUri'] = None # unassign
    sp = ov.server_profiles.update(resource=sp, id_or_uri=sp["uri"])
    
server = ov.server_hardware.get_by('name', new_server)[0]
# pprint(server)
if server['serverProfileUri'] is None:
    print("Assigning profile " + sp_name + " to hardware " + new_server)
    sp['serverHardwareUri'] = server['uri'] # assign
    sp = ov.server_profiles.update(resource=sp, id_or_uri=sp["uri"])
else:
    print("Server hardware " + new_server + " is already assigned to profile")

Profile CentOS-Golden is currently assigned will unassign it.
Powering Off the Server Profile: CentOS-Golden using ServerHardwareUri /rest/server-hardware/36343537-3338-4E43-3736-303230383036
Successfully changed the power state of 'CentOS-Golden' to Off
Assigning profile CentOS-Golden to hardware HPEDemo-POCframe01, bay 3


<br><br>
## Volumes <a name="Volumes"></a>

### Delete all Volumes <a name="Volume-deleteall"></a><br>
**WARNING**: This really deletes all volumes, actual command commented out.

In [ ]:
# get a list of all the storage templates
volumes = ov.volumes.get_all()
# pprint(volumes)

# delete them one-by-one
for volume in volumes: 
  print("Deleting the Volume: " + volume['name'] )
  # ov.volumes.delete(volume)

<br><br>
## Volume Templates <a name="VolumeTemplate"></a>

### Delete all Volume Templates <a name="VolumeTemplate-deleteall"></a><br>

In [ ]:
# get a list of all the storage templates
vol_temps = ov.storage_volume_templates.get_all()

# delete them one-by-one
for vol_temp in vol_temps: 
  print("Deleting the Volume Template: " + vol_temp['name'] )
  # ov.storage_volume_templates.delete(vol_temp)

<br><br>
## Logical Interconnect Groups <a name="LIG"></a>

### Add a network to an existing LIGs uplinkset <a name="LIG-uplink-network"></a><br>

In [ ]:
lig_name = "VC-LIG01"
lig_uplink_name = "Test"
eth_name = "VLAN400_10.2"

# Get the network
eth = ov.ethernet_networks.get_by('name', eth_name)[0]

# Get Logical Interconnect Group by property
lig = ov.logical_interconnect_groups.get_by('name', lig_name)[0]

lig_uplinksets = lig['uplinkSets']
for uplinkset in lig_uplinksets:
    #pprint(uplinkset)
    if uplinkset['name'] == lig_uplink_name:
        uplinkset['networkUris'].append(eth['uri'])
        ov.logical_interconnect_groups.update(lig)
    else:
        print("Could not find the uplinkset (by_name)")     

"""   
# Get an uplink set resource by name
print("\nGet uplink set by name")
uplink_set = ov.uplink_sets.get_by('name', 'Test')
pprint(uplink_set)

# Add an ethernet network to the uplink set
# To run this example you must define an ethernet network uri or ID below
ethernet_network_id = None
if ethernet_network_id:
    print("\nAdd an ethernet network to the uplink set")
    uplink_set = ov.uplink_sets.add_ethernet_networks(created_uplink_set['uri'], ethernet_network_id)
    print("The uplink set with name = '{name}' have now the networkUris:\n {networkUris}".format(**uplink_set))

"""

<br><br>
## Appliances <a name="Appliances"></a>

### eFuse an Appliance in bay 1/2 example <a name="Appliance-efuse"></a><br>
Table from 4.1 API spec on how to target different resources within the enclosure.<br>
http://h17007.www1.hpe.com/docs/enterprise/servers/oneview4.1/cic-api/en/index.html#rest/enclosures
<img src="images/EnclosurePatchTable.PNG" height="600" width="800" align="left">

In [ ]:
# WARNINIG: Make sure you target the correct enclosure as well as the operation you want! Pay attention to the table above
#           The actual patch request is commented out below for safety.
#           This is not something you normally do, only on request from support

enc_name = "HPEDemo-POCframe01"

# Get the Enclosure
efuse_enc = ov.enclosures.get_by('name', enc_name)

# If needed pretty print the entire JSON 
#pprint (efuse_enc)

# Grab the first enclosure uri
enc_uri = efuse_enc[0]['uri']

# Setup your patch request
patch_op = "replace"
#patch_path = "/applianceBays/1/bayPowerState"  # In a single POC frame this would be the Composer i.e Bay1
patch_path = "/applianceBays/2/bayPowerState"   # In a single POC frame this would be the Image Streamer i.e Bay2
patch_value = "E-Fuse" # Not the same for all paths

print("Request a PATCH on resource " + enc_uri + " using " + "op=" + patch_op + " path=" + patch_path + " val=" + patch_value)
#patched_end = ov.enclosures.patch(enc_uri, patch_op, patch_path, patch_value)


<br><br>
### eFuse a compute module in bay6 in a particular frame <a name="Appliance-efuse-compute"></a><br>

In [ ]:
# WARNINIG: Make sure you target the correct enclosure as well as the operation you want! Pay attention to the table above
#           The actual patch is commented out below for safety.
#           This is not something you normally do, only on request from support

enc_name = "HPEDemo-POCframe01"

# Get the Enclosure
efuse_enc = ov.enclosures.get_by('name', enc_name)

# If needed pretty print the entire JSON 
#pprint (efuse_enc)

# Grab the first enclosure uri
enc_uri = efuse_enc[0]['uri']

# Setup your patch request
patch_op = "replace"
patch_path = "/deviceBays/6/bayPowerState"   # In a single POC frame this would be the Computemodule in Bay6
patch_value = "E-Fuse" # Not the same for all paths

print("Request a PATCH on resource " + enc_uri + " using " + "op=" + patch_op + " path=" + patch_path + " val=" + patch_value)
#patched_end = ov.enclosures.patch(enc_uri, patch_op, patch_path, patch_value)